### Importações e Configurações

In [1]:
import numpy as np
import pandas as pd

In [ ]:
# Caminho para Salvar os Dados Limpos

path_data = './outputs/data/'

### Carregamento dos Dados Limpos

In [ ]:
df = pd.read_csv('../2_data_cleaning/outputs/data/df_clean.csv', parse_dates=['Date'], delimiter=',')
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size,Weekly_Sales_log
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,A,151315,10.123647
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,True,A,151315,10.737277
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,False,A,151315,10.635773
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,False,A,151315,9.873262
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,False,A,151315,9.990990


A partir deste notebook, o trabalho é feito exclusivamente sobre a versão limpa dos dados. Isso garante que as transformações criadas aqui não sejam influenciadas por inconsistências já tratadas anteriormente.

### Criando Variáveis Temporais

In [ ]:
# Variáveis Temporais Básicas

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['week'] = df['Date'].dt.isocalendar().week.astype(int)

In [5]:
# Variáveis Temporais Sazonais

df['week_sin'] = np.sin(2 * np.pi * df['week'] / 52)
df['week_cos'] = np.cos(2 * np.pi * df['week'] / 52)

Variáveis temporais explícitas permitem que modelos não temporais capturem padrões sazonais, e a codificação cíclica da semana do ano garante a continuidade periódica sem rupturas artificiais.

In [6]:
# Feriados e Interações

df['is_holiday'] = df['IsHoliday_x'].astype(int)
df['holiday_week'] = df['is_holiday'] * df['week']

As interações envolvendo feriados permitem capturar não apenas a presença do evento, mas também sua posição ao longo do ano, o que pode influenciar a magnitude do impacto nas vendas.

In [7]:
# Variáveis Janela Andante (Custo Prazo)

df = df.sort_values(['Store', 'Dept', 'Date'])
df['rolling_mean_4'] = (
    df.groupby(['Store', 'Dept'])['Weekly_Sales']
    .transform(lambda x: x.rolling(4, min_periods=1).mean())
)

Médias móveis capturam dependência temporal de curto prazo, permitindo que modelos estáticos incorporem informações do histórico recente de vendas.

In [8]:
# Lags

df['lag_1'] = (
    df.groupby(['Store', 'Dept'])['Weekly_Sales']
    .shift(1)
)
df['lag_52'] = (
    df.groupby(['Store', 'Dept'])['Weekly_Sales']
    .shift(52)
)

Variáveis de defasagem introduzem memória explícita no modelo, permitindo capturar tanto efeitos imediatos quanto padrões sazonais anuais.

In [9]:
# Tratamento de NaN Gerados Pelos Lags

df = df.dropna(subset=['lag_1', 'lag_52'])

As primeiras observações de cada série não possuem histórico suficiente para o cálculo das defasagens e, portanto, são removidas para manter consistência no conjunto de treino.

### Encoding de Variáveis Categoricas

In [10]:
df = pd.get_dummies(
    df,
    columns=['Type'],
    drop_first=True
)

O encoding é aplicado apenas a variáveis categóricas de baixa cardinalidade, evitando explosão dimensional e instabilidade nos coeficientes da regressão linear.

### Salvamento do Dados

In [11]:
df.to_csv('../data/processed/df_features.csv', index=False)